# ETL with PySpark using SQL API

In [1]:
try:
    import configparser
except ImportError:
    from six.moves import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')

# os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
# os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

['dl.cfg']

In [3]:
os.environ['AWS_ACCESS_KEY_ID']=config.get('AWS','AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY']=config.get('AWS','AWS_SECRET_ACCESS_KEY')

In [4]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

In [5]:
input_data = "s3a://udacity-dend/"

In [6]:
spark

In [7]:
sc = spark.sparkContext
#sc._jsc.hadoopConfiguration().set("fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", os.environ['AWS_ACCESS_KEY_ID'])
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", os.environ['AWS_SECRET_ACCESS_KEY'])

In [8]:
song_data = os.path.join(input_data, 'song_data/A/A/A/*.json')
df = spark.read.json(song_data)

In [10]:
log_data = os.path.join(input_data, 'log_data/*/*/*.json')
df_log = spark.read.json(log_data)

In [9]:
df.limit(5).toPandas()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARTC1LV1187B9A4858,51.45360,"Goldsmith's College, Lewisham, Lo",-0.01802,The Bonzo Dog Band,301.40036,1,SOAFBCP12A8C13CC7D,King Of Scurf (2007 Digital Remaster),1972
1,ARA23XO1187B9AF18F,40.57885,"Carteret, New Jersey",-74.21956,The Smithereens,192.52200,1,SOKTJDS12AF72A25E5,Drown In My Own Tears (24-Bit Digitally Remast...,0
2,ARSVTNL1187B992A91,51.50632,"London, England",-0.12714,Jonathan King,129.85424,1,SOEKAZG12AB018837E,I'll Slap Your Face (Entertainment USA Theme),2001
3,AR73AIO1187B9AD57B,37.77916,"San Francisco, CA",-122.42005,Western Addiction,118.07302,1,SOQPWCR12A6D4FB2A3,A Poor Recipe For Civic Cohesion,2005
4,ARXQBR11187B98A2CC,NaN,"Liverpool, England",NaN,Frankie Goes To Hollywood,821.05424,1,SOBRKGM12A8C139EF6,Welcome to the Pleasuredome,1985


In [12]:
df_log.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
3,None,Logged In,Wyatt,M,0,Scott,NaN,free,"Eureka-Arcata-Fortuna, CA",GET,Home,1.540872e+12,563,None,200,1542247071796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
4,None,Logged In,Austin,M,0,Rosales,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.541060e+12,521,None,200,1542252577796,Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20...,12


In [13]:
df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [14]:
df_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [15]:
df.count()

24

In [16]:
df_log.count()

8056

# Song Data

### Create Songs_Data `TempView`

In [17]:
df.createOrReplaceTempView("songs_data")

**`Songs` Query**

In [18]:
songs = spark.sql("""
select distinct
song_id,
title,
artist_id,
year,
duration 
from songs_data
""")

In [19]:
songs.show(5)

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOHOZBI12A8C132E3C|         Smash It Up|AR0MWD61187B9B2B12|2000|195.39546|
|SOXZYWX12A6310ED0C|     It's About Time|ARC1IHZ1187FB4E920|   0| 246.9873|
|SOHKNRJ12A6701D1F8|        Drop of Rain|AR10USD1187B99F3F1|   0|189.57016|
|SOIGICF12A8C141BC5|        Game & Watch|AREWD471187FB49873|2004|580.54485|
|SOAPERH12A58A787DC|The One And Only ...|ARZ5H0P1187B98A1DD|   0|230.42567|
+------------------+--------------------+------------------+----+---------+
only showing top 5 rows



### Write as Parquet file and Partition by Year and Artist_Id

In [24]:
#songs.write.partitionBy("year", "artist_id").parquet("s3a://atwan-udacity/p4/songs", mode='overwrite')

**`Artist` Query**

In [20]:
artist = spark.sql("""
select distinct
artist_id ,
artist_name ,
artist_location ,
artist_latitude ,
artist_longitude 
from songs_data
""")

In [21]:
artist.show(5)

+------------------+-----------------+---------------+---------------+----------------+
|         artist_id|      artist_name|artist_location|artist_latitude|artist_longitude|
+------------------+-----------------+---------------+---------------+----------------+
|ARJNIUY12298900C91|     Adelitas Way|               |           null|            null|
|AR5LMPY1187FB573FE|Chaka Khan_ Rufus|    Chicago, IL|       41.88415|       -87.63241|
|AR1C2IX1187B99BF74|  Broken Spindles|               |           null|            null|
|ARC1IHZ1187FB4E920|     Jamie Cullum|               |           null|            null|
|ARKYKXP11F50C47A6A| The Supersuckers|               |           null|            null|
+------------------+-----------------+---------------+---------------+----------------+
only showing top 5 rows



In [27]:
#artist.write.parquet("s3a://atwan-udacity/p4/artist", mode='overwrite')

# Log Data

Filter data for `NextSong` to indicate song plays

In [22]:
df_log = df_log.filter(df_log.page == 'NextSong')

### Create log_data `TempView`

In [23]:
df_log.createOrReplaceTempView("log_data")

In [24]:
# test
spark.sql("SELECT month('2016-07-30')").show()

+-------------------------------+
|month(CAST(2016-07-30 AS DATE))|
+-------------------------------+
|                              7|
+-------------------------------+



**note:** for checking conversion we can use Epoch Converter tool [here](https://www.epochconverter.com)

**`Time` Query**

In [25]:
spark.sql("""
select ts, 
from_unixtime(ts/1000, "hh:mm:ss") as start_time,
from_unixtime(ts/1000, "yyyy-MM-dd") as date,
month(from_unixtime(ts/1000, "yyyy-MM-dd")) as month,
year(from_unixtime(ts/1000, "yyyy-MM-dd")) as year,
day(from_unixtime(ts/1000, "yyyy-MM-dd")) as day,
weekofyear(from_unixtime(ts/1000, "yyyy-MM-dd")) as week,
hour(from_unixtime(ts/1000, "hh:mm:ss")) as hour,
weekday(from_unixtime(ts/1000, "yyyy-MM-dd")) as weekday
from log_data
where log_data.page = 'NextSong'
""").show(5)

+-------------+----------+----------+-----+----+---+----+----+-------+
|           ts|start_time|      date|month|year|day|week|hour|weekday|
+-------------+----------+----------+-----+----+---+----+----+-------+
|1542241826796|  12:30:26|2018-11-15|   11|2018| 15|  46|  12|      3|
|1542242481796|  12:41:21|2018-11-15|   11|2018| 15|  46|  12|      3|
|1542242741796|  12:45:41|2018-11-15|   11|2018| 15|  46|  12|      3|
|1542253449796|  03:44:09|2018-11-15|   11|2018| 15|  46|   3|      3|
|1542260935796|  05:48:55|2018-11-15|   11|2018| 15|  46|   5|      3|
+-------------+----------+----------+-----+----+---+----+----+-------+
only showing top 5 rows



**`User` Query**

In [26]:
user = spark.sql("""
SELECT distinct
cast(f.userid as smallint) as id,
f.firstname,
f.lastname,
f.gender,
f.level
from
    (SELECT
    se.userid,
    se.firstname ,
    se.lastname ,
    se.gender,
    se.level,
    se.ts
    from
    log_data se
    join(
        SELECT
        userid,
        max(ts) as mts
        from
        log_data
        where
        userid != ''
        group by 1
    ) level_latest on
    se.userid = level_latest.userid
    and se.ts = level_latest.mts
) f
""").show(5)

+---+---------+--------+------+-----+
| id|firstname|lastname|gender|level|
+---+---------+--------+------+-----+
| 26|     Ryan|   Smith|     M| free|
| 15|     Lily|    Koch|     F| paid|
|  8|   Kaylee| Summers|     F| free|
| 17| Makinley|   Jones|     F| free|
| 82|    Avery|Martinez|     F| paid|
+---+---------+--------+------+-----+
only showing top 5 rows



**`songplays` Query**

In [27]:
song_plays = spark.sql("""
select distinct
from_unixtime(ts/1000, "hh:mm:ss") as start_time,
cast(sev.userid as int) as id,
sev.level,
son.song_id ,
son.artist_id ,
sev.sessionid,
sev.location,
sev.useragent
from log_data sev
left join songs_data son 
on sev.artist = son.artist_name 
	and sev.length = son.duration
	and sev.song = son.title
where sev.userid != '' 
	and sev.page = 'NextSong'
""").show(5)

+----------+---+-----+-------+---------+---------+--------------------+--------------------+
|start_time| id|level|song_id|artist_id|sessionid|            location|           useragent|
+----------+---+-----+-------+---------+---------+--------------------+--------------------+
|  10:07:36| 30| paid|   null|     null|      324|San Jose-Sunnyval...|Mozilla/5.0 (Wind...|
|  03:37:16| 30| paid|   null|     null|      324|San Jose-Sunnyval...|Mozilla/5.0 (Wind...|
|  05:51:24| 97| paid|   null|     null|      605|Lansing-East Lans...|"Mozilla/5.0 (X11...|
|  09:18:34| 44| paid|   null|     null|      619|Waterloo-Cedar Fa...|Mozilla/5.0 (Maci...|
|  08:31:54| 88| paid|   null|     null|      744|Sacramento--Rosev...|"Mozilla/5.0 (Mac...|
+----------+---+-----+-------+---------+---------+--------------------+--------------------+
only showing top 5 rows

